This Code will explore the .nc data from Axial Fetch

In [1]:
# Import necessary libraries
import xarray as xr
import pandas as pd
import netCDF4 as nc
import numpy as np

/Users/earthnote/opt/anaconda3/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [2]:
cd /Users/earthnote/Desktop/AZA/2022/9

/Users/earthnote/Desktop/AZA/2022/9


In [3]:
ls

2022-09-01.local.nc*  2022-09-07.local.nc*  2022-09-19.local.nc*
2022-09-01.remote.nc* 2022-09-08.local.nc*  2022-09-20.local.nc*
2022-09-02.local.nc*  2022-09-09.local.nc*  2022-09-21.local.nc*
2022-09-02.remote.nc* 2022-09-10.local.nc*  2022-09-22.local.nc*
2022-09-03.local.nc*  2022-09-11.local.nc*  2022-09-23.local.nc*
2022-09-03.remote.nc* 2022-09-12.local.nc*  2022-09-24.local.nc*
2022-09-04.local.nc*  2022-09-13.local.nc*  2022-09-25.local.nc*
2022-09-04.remote.nc* 2022-09-14.local.nc*  2022-09-26.local.nc*
2022-09-05.local.nc*  2022-09-15.local.nc*  2022-09-27.local.nc*
2022-09-05.remote.nc* 2022-09-16.local.nc*  2022-09-28.local.nc*
2022-09-06.local.nc*  2022-09-17.local.nc*  2022-09-29.local.nc*
2022-09-06.remote.nc* 2022-09-18.local.nc*  2022-09-30.local.nc*


In [188]:
# Open the .nc file
dataset = nc.Dataset('2022-09-01.remote.nc')

In [189]:
if 'address' in dataset.variables:
    print("The dataset has the 'address' variable.")

    # Number of time points associated with each address
    time_points_per_address = len(time)

    
    variables = ['pitch', 'roll', 'temperature', 'sound_velocity', 'pressure', 'secondary_pressure', 
                  'range_delay', 'turn_around_time', 'sens_response_timestamp', 'si_response_timestamp', 'cs_response_timestamp']
    
    
    # Create a dictionary of DataFrames, one for each address
    dfs = {}
    # Extract the variables
    address = dataset.variables['address'][:]
    time = dataset.variables['time'][:]
    pitch = dataset.variables['pitch'][:]
    roll = dataset.variables['roll'][:]
    temperature = dataset.variables['temperature'][:]
    sound_velocity = dataset.variables['sound_velocity'][:]
    pressure = dataset.variables['pressure'][:]
    secondary_pressure = dataset.variables['secondary_pressure'][:]
    range_delay = dataset.variables['range_delay'][:]
    turn_around_time = dataset.variables['turn_around_time'][:]
    sens_response_timestamp = dataset.variables['sens_response_timestamp'][:]
    si_response_timestamp = dataset.variables['si_response_timestamp'][:]
    cs_response_timestamp = dataset.variables['cs_response_timestamp'][:]

    for i, addr in enumerate(address):
        df_addr = pd.DataFrame({
            'address': addr,
            'time': time.data,
            'pitch': pitch.data[i],
            'roll': roll.data[i],
            'temperature': temperature.data[i],
            'sound_velocity': sound_velocity.data[i],
            'pressure': pressure.data[i],
            'secondary_pressure': secondary_pressure.data[i],
            'range_delay0': range_delay.data[i][0],
            'range_delay1': range_delay.data[i][1],
            'range_delay2': range_delay.data[i][2],
            'range_delay3': range_delay.data[i][3],
            'turn_around_time': turn_around_time.data[i],
            'sens_response_timestamp': sens_response_timestamp.data[i],
            'si_response_timestamp': si_response_timestamp.data[i],
            'cs_response_timestamp': cs_response_timestamp.data[i]
        })
        dfs[addr] = df_addr
        
else:

    print("The dataset does not have the 'address' variable.")
    # Create a dictionary to hold the data
    data = {}
    # Loop over all variables
    for var in dataset.variables:
        data[var] = dataset.variables[var][:]

    # Create the DataFrame
    df = pd.DataFrame(data)

The dataset has the 'address' variable.


In [190]:
dfs[2501]

,address,time,pitch,roll,temperature,sound_velocity,pressure,secondary_pressure,range_delay0,range_delay1,range_delay2,range_delay3,turn_around_time,sens_response_timestamp,si_response_timestamp,cs_response_timestamp
0,2501,1661996899,8.300000e-01,2.260000e+00,2.440000e+00,1.484570e+03,6.690000e+01,1.552180e+04,NaN,2376916.0,NaN,NaN,80.0,1661996899,1661996904,1661996858
1,2501,1661996899,8.300000e-01,2.260000e+00,2.440000e+00,1.484570e+03,6.690000e+01,1.552180e+04,NaN,2376916.0,NaN,NaN,80.0,1661996899,1661996904,1661996858
2,2501,1661997616,8.300000e-01,2.250000e+00,2.440000e+00,1.484569e+03,6.690000e+01,1.552090e+04,NaN,2376942.0,NaN,NaN,80.0,1661997616,1661997621,1661996858
3,2501,1662001215,8.400000e-01,2.260000e+00,2.440000e+00,1.484563e+03,6.690000e+01,1.551590e+04,NaN,2376979.0,NaN,NaN,80.0,1662001215,1662001219,1661996858
4,2501,1662004814,8.300000e-01,2.250000e+00,2.450000e+00,1.484568e+03,6.690000e+01,1.551200e+04,NaN,2377009.0,NaN,NaN,80.0,1662004814,1662004819,1661996858
5,2501,1662008420,8.400000e-01,2.260000e+00,2.440000e+00,1.484554e+03,6.690000e+01,1.551030e+04,NaN,2377014.0,NaN,NaN,80.0,1662008420,1662008424,1661996858
6,2501,1662008420,8.400000e-01,2.260000e+00,2.440000e+00,1.484554e+03,6.690000e+01,1.551030e+04,NaN,2377014.0,NaN,NaN,80.0,1662008420,1662008424,1661996858
7,2501,1662011866,8.400000e-01,2.250000e+00,2.440000e+00,1.484549e+03,6.690000e+01,1.551050e+04,NaN,2377020.0,2425061.0,NaN,80.0,1662011866,1662011871,1662011818
8,2501,1662012006,8.400000e-01,2.250000e+00,2.440000e+00,1.484550e+03,6.690000e+01,1.551060e+04,NaN,2377025.0,2425082.0,NaN,80.0,1662012006,1662012023,1662011818
9,2501,1662012006,8.400000e-01,2.250000e+00,2.440000e+00,1.484550e+03,6.690000e+01,1.551060e+04,NaN,2377025.0,2425082.0,NaN,80.0,1662012006,1662012023,1662011818
